In [9]:
import socket

def request_image_capture():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    host = '172.20.10.4'  # Replace with your server's IP address
    port = 9999
    client_socket.connect((host, port))
    print("Connected to server.")
    
    client_socket.send('capture_image'.encode('utf-8'))
    response = client_socket.recv(1024).decode('utf-8')
    print(response)
    client_socket.close()

def shutdown_server():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    host = '172.20.10.4'  # Replace with your server's IP address
    port = 9999
    client_socket.connect((host, port))
    print("Connected to server.")
    
    client_socket.send('shutdown_server'.encode('utf-8'))
    client_socket.close()
    print("Sent shutdown signal to server.")


In [15]:
import opentrons.execute
from opentrons import protocol_api, types

## insures that robot is correctly initialized
!systemctl stop opentrons-robot-server

# Setup parameters
reservoir_name = 'nest_1_reservoir_195ml'
reservoir_location = 2
tiprack_name = 'opentrons_96_tiprack_300ul'
tiprack_location = 10
pipette_type = 'p300_multi'
repeat_times = 2

# Protocol initialization
protocol = opentrons.execute.get_protocol_api('2.11')

# Labware setup
reservoir = protocol.load_labware(reservoir_name, reservoir_location)
tiprack = protocol.load_labware(tiprack_name, tiprack_location)

# Pipette setup
pipette = protocol.load_instrument(pipette_type, 'right', tip_racks=[tiprack])

# Initialize and home
protocol.set_rail_lights(True)
protocol.home()
pipette.drop_tip()
print('Robot initialized, lights on, and homed.')

for i in range(repeat_times):  # Loop repeat_times int
    print(f'Starting loop {i+1}')
    pipette.pick_up_tip(tiprack.wells()[0])  # Pick up tips from the first row
    print('Picked up tips.')

    pipette.aspirate(150, reservoir.wells()[0])  # Aspirate 20ul from reservoir
    print('Aspirated 150ul.')

    # Move and hold for 2 seconds
    pipette.move_to(protocol.deck.position_for(6).move(types.Point(x=100, y=110, z=130)))
    print('Moved to position 6.')
    # request_image_capture()
    protocol.delay(seconds=2)
    print('Held position for 2 seconds.')

    # Dispense and return tips
    pipette.dispense(150, reservoir.wells()[0])
    print('Dispensed 20ul back to reservoir.')
    pipette.return_tip()
    print('Dropped the tips back.\n')
#protocol.home()
pipette.drop_tip()
protocol.set_rail_lights(False)
#shutdown_server()
print('Process completed. Lights turned off, and arm returned to home.\n\n')


Out of bounds move: X=(418.00000000000006 motor controller, 416.61520744316886 deck) too high for limit 418.0
Out of bounds move: X=(418.00000000000006 motor controller, 416.61520744316886 deck) too high for limit 418.0
Out of bounds move: X=(418.00000000000006 motor controller, 416.61520744316886 deck) too high for limit 418.0


Robot initialized, lights on, and homed.
Starting loop 1
Picked up tips.
Aspirated 150ul.
Moved to position 6.
Connected to server.
HD Image captured: default_project_20231221194005_hd.jpg
Held position for 2 seconds.
Dispensed 20ul back to reservoir.
Dropped the tips back.

Starting loop 2
Picked up tips.
Aspirated 150ul.
Moved to position 6.
Connected to server.
HD Image captured: default_project_20231221194034_hd.jpg
Held position for 2 seconds.
Dispensed 20ul back to reservoir.
Dropped the tips back.

Connected to server.
Sent shutdown signal to server.
Process completed. Lights turned off, and arm returned to home.




In [41]:
shutdown_server()

Connected to server.
Sent shutdown signal to server.


In [45]:
request_image_capture()

Connected to server.
HD Image captured: default_project_20231221175145_hd.jpg
